<a href="https://colab.research.google.com/github/pratik-poudel/jane-street/blob/main/4%20feautres_corr_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download gdonchyts/jane-street-market-prediction-parquet -f train.parquet
import zipfile
zip_ref = zipfile.ZipFile('/content/train.parquet.zip', 'r')
zip_ref.extractall('files')
zip_ref.close()

 99% 1.06G/1.08G [00:09<00:00, 76.8MB/s]
100% 1.08G/1.08G [00:09<00:00, 116MB/s] 


In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import gc
# pd.options.display.float_format| = "{:.2f}".format
pd.options.display.max_columns = 500

In [91]:
train = pd.read_parquet('/content/files/train.parquet')
train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]
train.shape

(1571415, 138)

In [92]:
original_features = [f for f in train.columns if 'feature' in f]

In [94]:
val_range = train[original_features].max()-train[original_features].min()
filler = pd.Series(train[original_features].min()-0.01*val_range, index=original_features)

In [97]:
def fill_missing(df):
    df[original_features] = df[original_features].fillna(filler)
    return df  

train = fill_missing(train)

train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1571415 entries, 0 to 1862595
Columns: 138 entries, date to ts_id
dtypes: float32(135), int64(3)
memory usage: 857.2 MB


In [98]:
def feature_transforms(df):
    # Generate Features using Linear shifting, Natural Logarithm and Square Root
    for f in [f'feature_{i}' for i in range(1,130)]: 
        # linear shifting to value above 1.0
        df['pos_'+str(f)] = (df[f]+abs(train[f].min())+1).astype(np.float16)
    for f in [f'feature_{i}' for i in range(1,130)]: 
        # Natural log of all the values
        df['log_'+str(f)] = np.log(df['pos_'+str(f)]).astype(np.float16)
    for f in [f'feature_{i}' for i in range(1,130)]: 
        # Square root of all the values
        df['sqrt_'+str(f)] = np.sqrt(df['pos_'+str(f)]).astype(np.float16)
    
    # Linearly shifted values are used for log and sqrt transformations
    # However they are useless since we have our original values which are 100% correlated
    # Let's drop them from our data
    df.drop([f'pos_feature_{i}' for i in range(1,130)], inplace=True, axis=1)
    return df

In [99]:
train_org = train.copy()

In [100]:
train = feature_transforms(train)
train.shape

(1571415, 396)

In [101]:
def other_trans(df):
    for i in original_features[1:]:
        df['quad_'+i] = np.square(df[i])
        df['cub_'+i] = np.power(df[i], 3)
    return df

In [102]:
train = other_trans(train)

In [103]:
train.shape    

(1571415, 654)

In [104]:
cubic_features = [f for f in train.columns if 'cub_' in f]
quad_features = [f for f in train.columns if 'quad_' in f]
sqrt_features = [f for f in train.columns if 'sqrt_' in f]
log_features = [f for f in train.columns if 'log_' in f]

In [105]:
len(cubic_features), len(quad_features), len(sqrt_features), len(log_features), len(original_features)

(129, 129, 129, 129, 130)

In [106]:
all_features = [f for f in train.columns if 'feature' in f]

In [107]:
corr_resp = train[all_features].corrwith(train['resp']).abs().sort_values(ascending=False).reset_index()
corr_resp3 = train[all_features].corrwith(train['resp_3']).abs().sort_values(ascending=False).reset_index()

In [108]:
corr_resp.columns = ['features', 'score_resp']
corr_resp3.columns = ['features', 'score_resp3']

In [109]:
corr_resp.to_csv('corr_resp.csv', index=False)
corr_resp3.to_csv('corr_resp3.csv', index=False)

In [122]:
org = train_org[original_features].corrwith(train['resp']).abs().sort_values(ascending=False)

In [125]:
to_combine = corr_resp[corr_resp['score_resp'] > 0.02]['features'].to_list()

In [126]:
len(to_combine)

18

In [130]:
three_comb = []
from itertools import combinations
for i in combinations(to_combine, 3):
    three_comb.append(i)

In [131]:
len(three_comb)

816

In [132]:
three_comb[0]

('cub_feature_39', 'feature_39', 'log_feature_39')

In [134]:
from itertools import permutations,combinations_with_replacement, combinations
operators = ["+","-","*","/"]
for i in combinations_with_replacement(operators, 2):
    print(i)

('+', '+')
('+', '-')
('+', '*')
('+', '/')
('-', '-')
('-', '*')
('-', '/')
('*', '*')
('*', '/')
('/', '/')


In [136]:
%%time
from google.colab import output

th = 0.02
feature_name =[]
score = []
count = 0
for i in three_comb:
    a = i[0]
    b = i[1]
    c = i[2]
    
    train['computed'] = train[a] + train[b] + train[c]
    cor = np.abs(train['computed'].corr(train['resp']))
    if cor > th:
        f_name = '+'.join([a, b, c])
        feature_name.append(f_name)
        score.append(cor)
    
    train['computed'] = train[a] + train[b] - train[c]
    cor = np.abs(train['computed'].corr(train['resp']))
    if cor > th:
        f_name = '+'.join([a, b])+"-"+str(c)
        feature_name.append(f_name)
        score.append(cor)
    
    train['computed'] = train[a] + train[b] * train[c]
    cor = np.abs(train['computed'].corr(train['resp']))
    if cor > th:
        f_name = '+'.join([a, b])+"*"+str(c)
        feature_name.append(f_name)
        score.append(cor)
    
    
    train['computed'] = train[a] + train[b] / train[c]
    cor = np.abs(train['computed'].corr(train['resp']))
    if cor > th:
        f_name = '+'.join([a, b])+"/"+str(c)
        feature_name.append(f_name)
        score.append(cor)

    train['computed'] = train[a] - train[b] - train[c]
    cor = np.abs(train['computed'].corr(train['resp']))
    if cor > th:
        f_name = '-'.join([a, b])+"-"+str(c)
        feature_name.append(f_name)
        score.append(cor)
    
    train['computed'] = train[a] - train[b] * train[c]
    cor = np.abs(train['computed'].corr(train['resp']))
    if cor > th:
        f_name = '-'.join([a, b])+"*"+str(c)
        feature_name.append(f_name)
        score.append(cor)
        
    train['computed'] = train[a] - train[b] / train[c]
    cor = np.abs(train['computed'].corr(train['resp']))
    if cor > th:
        f_name = '-'.join([a, b])+"/"+str(c)
        feature_name.append(f_name)
        score.append(cor)
        
    train['computed'] = train[a] * train[b] * train[c]
    cor = np.abs(train['computed'].corr(train['resp']))
    if cor > th:
        f_name = '*'.join([a, b])+"*"+str(c)
        feature_name.append(f_name)
        score.append(cor)
        
    train['computed'] = train[a] * train[b] / train[c]
    cor = np.abs(train['computed'].corr(train['resp']))
    if cor > th:
        f_name = '*'.join([a, b])+"/"+str(c)
        feature_name.append(f_name)
        score.append(cor)
        
    train['computed'] = train[a] / train[b] / train[c]
    cor = np.abs(train['computed'].corr(train['resp']))
    if cor > th:
        f_name = '/'.join([a, b])+"/"+str(c)
        feature_name.append(f_name)
        score.append(cor)
   
    count += 1
    print(count, "done out of ", len(three_comb) )
    if count % 10 ==0:
        output.clear()

811 done out of  816
812 done out of  816
813 done out of  816
814 done out of  816
815 done out of  816
816 done out of  816
CPU times: user 7min 6s, sys: 13.7 s, total: 7min 20s
Wall time: 7min 5s


In [138]:
pd.DataFrame(zip(feature_name, score), columns=['features', 'score']).sort_values(by='score',ascending=False).to_csv('three_features.csv', index=False)

In [200]:
three_features = pd.read_csv('three_features.csv')

In [153]:
split = three_features[three_features['score'] > 0.063]['features'].to_list()

In [205]:
three_features[three_features['score'] > 0.064]

,features,score
0,cub_feature_39-cub_feature_37/sqrt_feature_29,0.068198
1,cub_feature_39-cub_feature_37/sqrt_feature_40,0.066402
2,cub_feature_39-feature_37*sqrt_feature_29,0.065228
3,cub_feature_39-sqrt_feature_40*feature_37,0.065133
4,cub_feature_39-sqrt_feature_37*feature_37,0.064913
5,cub_feature_39-sqrt_feature_39*feature_37,0.064905
6,cub_feature_39-feature_37*sqrt_feature_38,0.064860
7,cub_feature_39-feature_37*log_feature_40,0.064653
8,cub_feature_39+feature_29*sqrt_feature_29,0.064638
9,cub_feature_39-log_feature_37*feature_37,0.064556


In [146]:
import re
feature_split = [] 
symbol_split = []
all_name = []
for i in split:
    all_name.append(i)
    features_list = re.split('[- + * /]',i)
    maths = re.findall(r"([\+\-\*\/]+)", i)
    feature_split.append(features_list)
    symbol_split.append(maths + [0])

In [147]:
feature_split

[['cub_feature_39', 'cub_feature_37', 'sqrt_feature_29'],
 ['cub_feature_39', 'cub_feature_37', 'sqrt_feature_40'],
 ['cub_feature_39', 'feature_37', 'sqrt_feature_29'],
 ['cub_feature_39', 'sqrt_feature_40', 'feature_37']]

In [148]:
count = 0
for i,j in zip(feature_split, symbol_split):
#     print([train.columns.get_loc(i[index]) for index in range(len(i))])
#     print(len(i))
    feature_name = all_name[count]
    print("train['"+feature_name+"']", "= " "train["+"'"+i[0]+"'"+"]", str(j[0]), "train["+"'"+i[1]+"'"+"]", str(j[1]), "train["+"'"+i[2]+"'"+"]")
    count += 1

train['cub_feature_39-cub_feature_37/sqrt_feature_29'] = train['cub_feature_39'] - train['cub_feature_37'] / train['sqrt_feature_29']
train['cub_feature_39-cub_feature_37/sqrt_feature_40'] = train['cub_feature_39'] - train['cub_feature_37'] / train['sqrt_feature_40']
train['cub_feature_39-feature_37*sqrt_feature_29'] = train['cub_feature_39'] - train['feature_37'] * train['sqrt_feature_29']
train['cub_feature_39-sqrt_feature_40*feature_37'] = train['cub_feature_39'] - train['sqrt_feature_40'] * train['feature_37']


# Two Features

In [163]:
to_combine = corr_resp[corr_resp['score_resp'] > 0.005]['features'].to_list()

In [164]:
len(to_combine)

197

In [165]:
corr_resp.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645
features,cub_feature_39,feature_39,log_feature_39,sqrt_feature_39,cub_feature_37,log_feature_37,sqrt_feature_37,feature_40,sqrt_feature_40,feature_37,log_feature_40,feature_29,sqrt_feature_29,cub_feature_40,feature_0,cub_feature_38,log_feature_38,sqrt_feature_38,feature_38,feature_33,feature_35,feature_1,log_feature_29,log_feature_103,sqrt_feature_103,feature_53,sqrt_feature_1,feature_19,feature_69,feature_122,sqrt_feature_53,feature_124,feature_126,feature_103,sqrt_feature_122,feature_128,log_feature_91,sqrt_feature_124,log_feature_53,quad_feature_1,sqrt_feature_128,log_feature_97,log_feature_122,sqrt_feature_126,sqrt_feature_69,log_feature_85,feature_30,feature_9,sqrt_feature_91,log_feature_128,quad_feature_69,log_feature_1,log_feature_124,sqrt_feature_19,sqrt_feature_43,sqrt_feature_97,log_feature_43,feature_43,log_feature_126,sqrt_feature_85,sqrt_feature_42,quad_feature_53,feature_42,log_feature_42,sqrt_feature_33,feature_31,log_feature_69,sqrt_feature_9,feature_91,cub_feature_4,feature_120,log_feature_51,sqrt_feature_106,feature_25,sqrt_feature_30,feature_97,quad_feature_39,sqrt_feature_35,feature_5,quad_feature_122,log_feature_106,sqrt_feature_51,sqrt_feature_5,cub_feature_3,feature_73,feature_2,feature_106,feature_85,quad_feature_126,quad_feature_124,quad_feature_128,sqrt_feature_120,feature_51,log_feature_5,feature_13,sqrt_feature_2,quad_feature_5,cub_feature_53,feature_23,quad_feature_89,feature_101,feature_44,quad_feature_113,feature_71,log_feature_2,feature_83,feature_113,sqrt_feature_83,quad_feature_96,sqrt_feature_73,cub_feature_69,sqrt_feature_44,feature_79,feature_27,log_feature_88,log_feature_83,sqrt_feature_101,log_feature_19,feature_89,feature_50,quad_feature_95,sqrt_feature_50,feature_52,log_feature_120,quad_feature_101,sqrt_feature_88,log_feature_9,feature_21,feature_3,cub_feature_1,quad_feature_108,log_feature_94,sqrt_feature_52,log_feature_50,sqrt_feature_31,quad_feature_107,log_feature_100,quad_feature_119,sqrt_feature_71,feature_34,log_feature_44,log_feature_52,feature_15,cub_feature_122,cub_feature_126,sqrt_feature_3,cub_feature_96,sqrt_feature

In [166]:
two_comb = []
from itertools import combinations
for i in combinations(to_combine, 2):
    two_comb.append(i)

In [167]:
len(two_comb)

19306

In [168]:
two_comb[0]

('cub_feature_39', 'feature_39')

In [169]:
from itertools import permutations,combinations_with_replacement, combinations
operators = ["+","-","*","/"]
for i in combinations_with_replacement(operators, 1):
    print(i)

('+',)
('-',)
('*',)
('/',)


In [182]:
two_comb[:5]

[('cub_feature_39', 'feature_39'),
 ('cub_feature_39', 'log_feature_39'),
 ('cub_feature_39', 'sqrt_feature_39'),
 ('cub_feature_39', 'cub_feature_37'),
 ('cub_feature_39', 'log_feature_37')]

In [194]:
%%time
from google.colab import output

th = 0.02
feature_name =[]
score = []
count = 0
for i in two_comb:
    a = i[0]
    b = i[1]
    # c = i[2]
    
    train['computed'] = train[a] + train[b]
    cor = np.abs(train['computed'].corr(train['resp']))
    if cor > th:
        f_name = '+'.join([a, b])
        feature_name.append(f_name)
        score.append(cor)
    
    train['computed'] = train[a] - train[b]
    cor = np.abs(train['computed'].corr(train['resp']))
    if cor > th:
        f_name = '-'.join([a, b])
        feature_name.append(f_name)
        score.append(cor)
    
    train['computed'] = train[a] * train[b]
    cor = np.abs(train['computed'].corr(train['resp']))
    if cor > th:
        f_name = '*'.join([a, b])
        feature_name.append(f_name)
        score.append(cor)
    
    
    train['computed'] = train[a] / train[b]
    cor = np.abs(train['computed'].corr(train['resp']))
    if cor > th:
        f_name = '/'.join([a, b])
        feature_name.append(f_name)
        score.append(cor)

    train['computed'] = train[b] / train[a]
    cor = np.abs(train['computed'].corr(train['resp']))
    if cor > th:
        f_name = '/'.join([b, a])
        feature_name.append(f_name)
        score.append(cor)
   
    count += 1
    print(count, "done out of ", len(two_comb) )
    if count % 10 ==0:
        output.clear()

19301 done out of  19306
19302 done out of  19306
19303 done out of  19306
19304 done out of  19306
19305 done out of  19306
19306 done out of  19306
CPU times: user 1h 9min 28s, sys: 2min 26s, total: 1h 11min 55s
Wall time: 1h 10min 34s


In [196]:
two_features = pd.DataFrame(zip(feature_name, score), columns=['features', 'score']).sort_values(by='score',ascending=False)

In [197]:
two_features

,features,score
472,cub_feature_39*log_feature_41,0.063808
322,cub_feature_39/sqrt_feature_44,0.063706
52,cub_feature_39+feature_1,0.063581
496,cub_feature_39*sqrt_feature_41,0.063545
484,cub_feature_39/sqrt_feature_89,0.063487
...,...,...
2791,sqrt_feature_40+sqrt_feature_83,0.020005
5281,feature_38+log_feature_106,0.020004
4980,cub_feature_38+quad_feature_107,0.020004
5212,feature_38*log_feature_29,0.020002


In [198]:
two_features.to_csv('two_features.csv', index=False)

In [ ]:
# pd.DataFrame(zip(feature_name, score), columns=['features', 'score'])
# .sort_values(by='score',ascending=False)
# .to_csv('three_features.csv', index=False)

In [206]:
two_features[two_features['score'] > 0.063]

,features,score
472,cub_feature_39*log_feature_41,0.063808
322,cub_feature_39/sqrt_feature_44,0.063706
52,cub_feature_39+feature_1,0.063581
496,cub_feature_39*sqrt_feature_41,0.063545
484,cub_feature_39/sqrt_feature_89,0.063487
370,cub_feature_39-feature_3,0.063384
24,cub_feature_39-feature_37,0.063382
95,cub_feature_39/sqrt_feature_122,0.063362
74,cub_feature_39+feature_69,0.063343
420,cub_feature_39/sqrt_feature_113,0.063343
